# Exploring conformational space of selected macrocycles - "M1"; <br /> Part 1: Generation and selection of conformer candidates (MM methods)

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [173]:
import glob
import py3Dmol

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline 

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Mon May 29 12:57:48 2017


In [174]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def write_to_dict(prefix, suppl):
    moldict = {}
    for i, mol in enumerate(suppl):
        name = prefix + str(i)
        moldict[name] = mol
    return moldict

def align_structures_to_crystal(moldict):
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))

def align_structures_to_lowest_energy(moldict, energy_dict):
    """
    align structures from the structures' dictionary
    
    note that the dictionary in python is not ordered, so the "first" element is arbitrary
    """
    energy_sorted = sorted(energy_dict.items(), key=lambda x: x[1])
    first = energy_sorted[0][0]
    core_first = moldict[first].GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,moldict[first],atomMap=list(zip(core_mol,core_first)))        

def prepare_view(moldict):
    p = py3Dmol.view(width=400,height=400)
    for key, mol in moldict.items():
        mb = Chem.MolToMolBlock(mol)
        p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.15'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p        

def make_similarity_matrix(moldict):
    
    similarity_matrix = {}

    for k1, m1 in moldict.items():
        for k2, m2 in moldict.items():
            if (k1, k2) in similarity_matrix.keys() or (k2, k1) in similarity_matrix.keys():
                pass
            else:
                if k1 != k2:
                    rms = AllChem.GetBestRMS(Chem.RemoveHs(m1),Chem.RemoveHs(m2))
                    similarity_matrix[(k1, k2)] = rms 
                    
    return similarity_matrix


def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

## Crystal structure of "M1" macrocycle

In [175]:
cm1 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.xyz','r').read()
vcm1 = py3Dmol.view(width=400,height=400)
vcm1.removeAllModels()
vcm1.addModel(cm1,'xyz')
vcm1.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm1.setBackgroundColor('0xeeeeee')
vcm1.zoomTo()
vcm1.show()

In [176]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'
core_smiles = 'n1ccccc1'

m1 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.sdf')
m1_crystal = templ_m1[0]

## Conformers generated with the Balloon software:

Conformers were generated using the genetic algorithm as implemented in the Balloon software:

* starting with the crystal geometry kept as a template, results with prefix: "m1_b_sdf"; the crystal is of the "ss-ss" type;

* starting with the SMILES signature of M1 and allowing to "rebuild the geometry" (option --rebuildGeometry), results with prefix: "m1_b_smi"

* starting with structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

    where "ss\_sa" means "(syn-syn)\_(syn-anti)" configuration, etc. with the bracket notation used to mark conformations around the rings.
    

In all cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

In [177]:
inps_m1_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsdf/*.sdf')
inps_m1_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsmiles/*.sdf')
inps_m1_b_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_ss_sa/*.sdf')
inps_m1_b_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_ss_aa/*.sdf')
inps_m1_b_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_sa/*.sdf')
inps_m1_b_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_as/*.sdf')
inps_m1_b_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_aa/*.sdf')
inps_m1_b_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_aa_aa/*.sdf')

In [178]:
e_m1_b_sdf = grep_energies_from_sdf_outputs(inps_m1_b_sdf)
e_m1_b_smi = grep_energies_from_sdf_outputs(inps_m1_b_smi)
e_m1_b_ss_sa = grep_energies_from_sdf_outputs(inps_m1_b_ss_sa)
e_m1_b_ss_aa = grep_energies_from_sdf_outputs(inps_m1_b_ss_aa)
e_m1_b_sa_sa = grep_energies_from_sdf_outputs(inps_m1_b_sa_sa)
e_m1_b_sa_as = grep_energies_from_sdf_outputs(inps_m1_b_sa_as)
e_m1_b_sa_aa = grep_energies_from_sdf_outputs(inps_m1_b_sa_aa)
e_m1_b_aa_aa = grep_energies_from_sdf_outputs(inps_m1_b_aa_aa)

In [179]:
# write conformers to dictionaries
   
suppl_m1_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_sdfout.sdf')
suppl_m1_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_smilesout.sdf')
suppl_m1_b_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_ss_sa_sdfout.sdf')
suppl_m1_b_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_ss_aa_sdfout.sdf')
suppl_m1_b_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_sa_sdfout.sdf')
suppl_m1_b_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_as_sdfout.sdf')
suppl_m1_b_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_aa_sdfout.sdf')
suppl_m1_b_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_aa_aa_sdfout.sdf')

allmol_m1_b_sdf   = write_to_dict("m1_b_sdf_", suppl_m1_b_sdf)
allmol_m1_b_smi   = write_to_dict("m1_b_smi_", suppl_m1_b_smi)
allmol_m1_b_ss_sa = write_to_dict("m1_b_ss_sa_", suppl_m1_b_ss_sa)
allmol_m1_b_ss_aa = write_to_dict("m1_b_ss_aa_", suppl_m1_b_ss_aa)
allmol_m1_b_sa_sa = write_to_dict("m1_b_sa_sa_", suppl_m1_b_sa_sa)
allmol_m1_b_sa_as = write_to_dict("m1_b_sa_as_", suppl_m1_b_sa_as)
allmol_m1_b_sa_aa = write_to_dict("m1_b_sa_aa_", suppl_m1_b_sa_aa)
allmol_m1_b_aa_aa = write_to_dict("m1_b_aa_aa_", suppl_m1_b_aa_aa)

The generated conformer structures are presented in a separate notebook: [link](http://nbviewer.jupyter.org/github/gosiao/icho-notebooks/blob/master/conformers_m1_suppl1.ipynb).

### pre-screening

All the generated structures are pre-optimized with MM methods (MMFF94-like force field). To remove potential duplicates, we will:

* calculate the root-mean-square-distance (RMSD) between the pairs of conformers (taking into account the heavy atoms only);
* compare the energies of conformers from pairs which are found to be similar (RMSD lower than the threshold);
* if these energies are too similar (the difference lower than the threshold), we will remove the conformer which has the higher energy value;

In [180]:
allmol_m1_b = {}
allmol_m1_b.update(allmol_m1_b_sdf)
allmol_m1_b.update(allmol_m1_b_smi)
allmol_m1_b.update(allmol_m1_b_ss_sa)
allmol_m1_b.update(allmol_m1_b_ss_aa)
allmol_m1_b.update(allmol_m1_b_sa_sa)
allmol_m1_b.update(allmol_m1_b_sa_as)
allmol_m1_b.update(allmol_m1_b_sa_aa)
allmol_m1_b.update(allmol_m1_b_aa_aa)

print("The total number of generated conformers = ", len(allmol_m1_b))
             
energy_m1_b = {}
energy_m1_b.update(e_m1_b_sdf)
energy_m1_b.update(e_m1_b_smi)
energy_m1_b.update(e_m1_b_ss_sa)
energy_m1_b.update(e_m1_b_ss_aa)
energy_m1_b.update(e_m1_b_sa_sa)
energy_m1_b.update(e_m1_b_sa_as)
energy_m1_b.update(e_m1_b_sa_aa)
energy_m1_b.update(e_m1_b_aa_aa)

The total number of generated conformers =  94


In [181]:
# 1. calculate the similarity matrix between all pairs of conformers and sort its elements from the lowest
# (the most similar structures) to the largest values (the most different structures)
similarity_matrix_b = make_similarity_matrix(allmol_m1_b)
similarity_matrix_b_sorted = sorted(similarity_matrix_b.items(), key=lambda x: x[1])

# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than threshold ("similarity_thresh"), 
# compare energies; then if the energies are similar (controlled by the "energy_thresh"), 
#then remove the one with higher energy

to_be_deleted = []
similarity_thresh = 0.5 # Angstrom (same as Balloon's original RMSD threshold used for conformer generation)
energy_thresh     = 1 # kcal/mol (Balloon original threshold used for conformer generation is 25 kcal/mol)

for pair in similarity_matrix_b_sorted:
    if pair[1] < similarity_thresh:
        conf1 = pair[0][0]
        conf2 = pair[0][1]
        if abs(energy_m1_b[conf1] - energy_m1_b[conf2]) < energy_thresh:
            #print("conf1, conf2, E(conf1), E(conf2) = ", conf1, conf2, energy_m1_b[conf1], energy_m1_b[conf2])
            if energy_m1_b[conf1] < energy_m1_b[conf2]:
                to_be_deleted.append(conf2)
            else:
                to_be_deleted.append(conf1)

for mol in to_be_deleted:
    print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix_b.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix_b[k]
    allmol_m1_b.pop(mol, None)
    energy_m1_b.pop(mol, None)
    

to_be_deleted:  m1_b_ss_sa_8
to_be_deleted:  m1_b_sa_as_2
to_be_deleted:  m1_b_sa_sa_0
to_be_deleted:  m1_b_ss_sa_2
to_be_deleted:  m1_b_sa_as_1
to_be_deleted:  m1_b_sa_as_10
to_be_deleted:  m1_b_sa_as_6
to_be_deleted:  m1_b_sa_as_1
to_be_deleted:  m1_b_ss_sa_0
to_be_deleted:  m1_b_sa_sa_8
to_be_deleted:  m1_b_sa_aa_5
to_be_deleted:  m1_b_sa_sa_5
to_be_deleted:  m1_b_ss_sa_6
to_be_deleted:  m1_b_sa_sa_5
to_be_deleted:  m1_b_sa_sa_13
to_be_deleted:  m1_b_sa_as_0
to_be_deleted:  m1_b_smi_1
to_be_deleted:  m1_b_sa_sa_7
to_be_deleted:  m1_b_sa_aa_4
to_be_deleted:  m1_b_ss_sa_0
to_be_deleted:  m1_b_sa_as_3
to_be_deleted:  m1_b_sa_as_10
to_be_deleted:  m1_b_smi_1
to_be_deleted:  m1_b_sa_as_4
to_be_deleted:  m1_b_sdf_1
to_be_deleted:  m1_b_sa_sa_0
to_be_deleted:  m1_b_aa_aa_1
to_be_deleted:  m1_b_sa_aa_8
to_be_deleted:  m1_b_sa_sa_3
to_be_deleted:  m1_b_sdf_2
to_be_deleted:  m1_b_sdf_2
to_be_deleted:  m1_b_sa_sa_14
to_be_deleted:  m1_b_ss_sa_1
to_be_deleted:  m1_b_aa_aa_5
to_be_deleted:  m1_b

In [182]:
print("We have removed the potential conformer duplicates.")
print("The final number of remaining conformers = ", len(allmol_m1_b))
print("Below we present all the remaining conformers aligned (to one ring).")

We have removed the potential conformer duplicates.
The final number of remaining conformers =  66
Below we present all the remaining conformers aligned (to one ring).


In [183]:
align_structures_to_lowest_energy(allmol_m1_b, energy_m1_b)
p_b = prepare_view(allmol_m1_b)
p_b.show()

In [184]:
print("Sorted energy of all selected conformers and the energy differences with respect to the lowest:")
energy_sorted = sorted(energy_m1_b.items(), key=lambda x: x[1])
energy_diff = []
e_min = energy_sorted[0][1]
for e in energy_sorted:
    e_diff = e[1] - e_min
    energy_diff.append([e[0], e[1], e_diff])

e_df = pd.DataFrame(energy_diff, columns=["conformer", "E", "E - Emin"])
e_df

Sorted energy of all selected conformers and the energy differences with respect to the lowest:


,conformer,E,E - Emin
0,m1_b_sdf_0,53.045942,0.000000
1,m1_b_smi_0,53.057266,0.011324
2,m1_b_sdf_3,53.710066,0.664124
3,m1_b_smi_2,54.225857,1.179916
4,m1_b_smi_3,54.744909,1.698967
5,m1_b_sdf_4,55.067265,2.021323
6,m1_b_sdf_5,55.482657,2.436716
7,m1_b_sa_sa_1,55.548698,2.502756
8,m1_b_sdf_6,56.225813,3.179871
9,m1_b_sdf_7,56.622176,3.576235


## Conformers generated with the RDKit software:

Conformers were generated using the genetic algorithm as implemented in the RDKit software:

* starting with the crystal geometry kept as a template, results with prefix: "m1_b_sdf"; the crystal is of the "ss-ss" type;

* starting with the SMILES signature of M1 and allowing to "rebuild the geometry" (option --rebuildGeometry), results with prefix: "m1_b_smi"

* starting with structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

    where "ss\_sa" means "(syn-syn)\_(syn-anti)" configuration, etc. with the bracket notation used to mark conformations around the rings.
    

In all cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

In [195]:
inps_m1_rdkit_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_smiles/*.sdf')
inps_m1_rdkit_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_sdf/*.sdf')
inps_m1_rdkit_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_ss_sa/*.sdf')
inps_m1_rdkit_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_ss_aa/*.sdf')
inps_m1_rdkit_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_sa/*.sdf')
inps_m1_rdkit_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_as/*.sdf')
inps_m1_rdkit_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_aa/*.sdf')
inps_m1_rdkit_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_aa_aa/*.sdf')

In [196]:
e_m1_rdkit_smi = grep_energies_from_sdf_outputs(inps_m1_rdkit_smi)
e_m1_rdkit_sdf = grep_energies_from_sdf_outputs(inps_m1_rdkit_sdf)
e_m1_rdkit_ss_sa = grep_energies_from_sdf_outputs(inps_m1_rdkit_ss_sa)
e_m1_rdkit_ss_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_ss_aa)
e_m1_rdkit_sa_sa = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_sa)
e_m1_rdkit_sa_as = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_as)
e_m1_rdkit_sa_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_aa)
e_m1_rdkit_aa_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_aa_aa)

In [198]:
# write conformers to dictionaries
suppl_m1_rdkit_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_smiles.sdf')
suppl_m1_rdkit_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_sdf.sdf')
suppl_m1_rdkit_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_ss_sa.sdf')
suppl_m1_rdkit_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_ss_aa.sdf')
suppl_m1_rdkit_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_sa.sdf')
suppl_m1_rdkit_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_as.sdf')
suppl_m1_rdkit_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_aa.sdf')
suppl_m1_rdkit_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_aa_aa.sdf')

allmol_m1_rdkit_smi = write_to_dict("m1_rdkit_smi_", suppl_m1_rdkit_smi)
allmol_m1_rdkit_sdf = write_to_dict("m1_rdkit_sdf_", suppl_m1_rdkit_sdf)
allmol_m1_rdkit_ss_sa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_ss_sa)
allmol_m1_rdkit_ss_aa = write_to_dict("m1_rdkit_ss_aa_", suppl_m1_rdkit_ss_aa)
allmol_m1_rdkit_sa_sa = write_to_dict("m1_rdkit_sa_sa_", suppl_m1_rdkit_sa_sa)
allmol_m1_rdkit_sa_as = write_to_dict("m1_rdkit_sa_as_", suppl_m1_rdkit_sa_as)
allmol_m1_rdkit_sa_aa = write_to_dict("m1_rdkit_sa_aa_", suppl_m1_rdkit_sa_aa)
allmol_m1_rdkit_aa_aa = write_to_dict("m1_rdkit_aa_aa_", suppl_m1_rdkit_aa_aa)

In [199]:
# align:
align_structures_to_lowest_energy(allmol_m1_rdkit_sdf, e_m1_rdkit_sdf)
align_structures_to_lowest_energy(allmol_m1_rdkit_smi, e_m1_rdkit_smi)
align_structures_to_lowest_energy(allmol_m1_rdkit_ss_sa, e_m1_rdkit_ss_sa)
align_structures_to_lowest_energy(allmol_m1_rdkit_ss_aa, e_m1_rdkit_ss_aa)
align_structures_to_lowest_energy(allmol_m1_rdkit_sa_sa, e_m1_rdkit_sa_sa)
align_structures_to_lowest_energy(allmol_m1_rdkit_sa_as, e_m1_rdkit_sa_as)
align_structures_to_lowest_energy(allmol_m1_rdkit_sa_aa, e_m1_rdkit_sa_aa)
align_structures_to_lowest_energy(allmol_m1_rdkit_aa_aa, e_m1_rdkit_aa_aa)

In [ ]:
%%html
<table>
  <tr>
    <td id="m1_rdkit_crystal" ></td>
    <td id="m1_rdkit_smiles"  ></td>
  <tr>
    <td> m1_rdkit_crystal </td>
    <td> m1_rdkit_smiles  </td>  
  </tr>
</table>

In [ ]:
# view:
p1_rdkit_handles=[]

p1_rdkit_sdf = py3Dmol.view(width=400,height=400)
p1_rdkit_sdf.removeAllModels()
for key, mol in allmol_m1_rdkit_sdf.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_rdkit_sdf.addModel(mb,'sdf')    
p1_rdkit_sdf.setStyle({'stick':{'radius':'0.15'}})
p1_rdkit_sdf.setBackgroundColor('0xeeeeee')
p1_rdkit_sdf.zoomTo()
p1_rdkit_handles.append(p1_rdkit_sdf)

p1_rdkit_smi = py3Dmol.view(width=400,height=400)
p1_rdkit_smi.removeAllModels()
for key, mol in allmol_m1_rdkit_smi.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_rdkit_smi.addModel(mb,'sdf')    
p1_rdkit_smi.setStyle({'stick':{'radius':'0.15'}})
p1_rdkit_smi.setBackgroundColor('0xeeeeee')
p1_rdkit_smi.zoomTo()
p1_rdkit_handles.append(p1_rdkit_smi)

In [ ]:
p1_rdkit_handles[0].insert('m1_rdkit_crystal')

In [ ]:
p1_rdkit_handles[1].insert('m1_rdkit_smiles')

### pre-screening

In [ ]:
allmol_m1_rdkit = {}
allmol_m1_rdkit.update(allmol_m1_rdkit_sdf)
allmol_m1_rdkit.update(allmol_m1_rdkit_smi)

energy_m1_rdkit = {}
energy_m1_rdkit.update(e_m1_rdkit_sdf)
energy_m1_rdkit.update(e_m1_rdkit_smi)

rms_m1_rdkit = {}
for key, mol in allmol_m1_rdkit.items():
    rms_m1_rdkit[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m1_rdkit[key], rms_m1_rdkit[key]))

In [ ]:
rms_sorted = sorted(rms_m1_rdkit.items(), key=lambda x: x[1])
rms_thresh = 0.05

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m1_rdkit[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1_rdkit, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1_rdkit[mol]
    del energy_m1_rdkit[mol]
    del rms_m1_rdkit[mol]  

Below we will align the selected conformers:

In [ ]:
for key, mol in allmol_m1_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p_r = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1_rdkit.items():
    mb = Chem.MolToMolBlock(mol)
    p_r.addModel(mb,'sdf')
p_r.setStyle({'stick':{'radius':'0.15'}})
p_r.setBackgroundColor('0xeeeeee')
p_r.zoomTo()
p_r.show()

## Summary

Now let's generate a list of all conformers (from all programs used, as presented above). We can further pre-screen all the structures and remove potential duplicates. Here we can also use more crude RMSD threshold.

In [ ]:
allmol_m1 = {}
allmol_m1.update(allmol_m1_b)
allmol_m1.update(allmol_m1_rdkit)

energy_m1 = {}
energy_m1.update(energy_m1_b)
energy_m1.update(energy_m1_rdkit)

rms_m1 = {}
for key, mol in allmol_m1.items():
    rms_m1[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m1[key], rms_m1[key]))

Now we will calculate the similarity matrix, which shows the RMSD value for any two conformers. 

We will print the similarity matrix for pairs of conformers from the lowest value of the RMSD (= the most similar structures) to the highest values of the RMSD (= the least similar structures):

In [ ]:
similarity_matrix = {}

for k1, m1 in allmol_m1.items():
    for k2, m2 in allmol_m1.items():
        if (k1, k2) in similarity_matrix.keys() or (k2, k1) in similarity_matrix.keys():
            pass
        else:
            if k1 != k2:
                rms = AllChem.GetBestRMS(Chem.RemoveHs(m1),Chem.RemoveHs(m2))
                similarity_matrix[(k1, k2)] = rms 
    
similarity_matrix_sorted = sorted(similarity_matrix.items(), key=lambda x: x[1])

 
similarity_matrix_sorted

In [ ]:
rms_sorted = sorted(rms_m1.items(), key=lambda x: x[1])
rms_thresh = 0.1

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1[mol]
    del energy_m1[mol]
    del rms_m1[mol]  

Finally we can align all conformers which will further be used as starting points in DFT geometry optimizations:

In [ ]:
print(len(allmol_m1))

for key, mol in allmol_m1.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1.items():
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

Write the selected conformers' names to the list "list_selected_conformers_from_balloon_rdkit". It will be used to generate Gaussian inputs:

In [ ]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1/list_selected_conformers_from_ballon_rdkit", "w") as f:
    for key, mol in allmol_m1.items():
        f.write(key+"\n")

energy_sorted = sorted(energy_m1.items(), key=lambda x: x[1])
print(energy_sorted)
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1/detailed_list_selected_conformers_from_ballon_rdkit", "w") as f:
    for pair in energy_sorted:
        f.write("{0:30}   {1}\n".format(pair[0], pair[1]))        